In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
%matplotlib qt

In [19]:
mexico = pd.read_csv('data/covidmxraw.csv', skiprows=7)
mexico = mexico[:-3]
mexico = mexico[['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 2']]
mexico = mexico.rename(columns={'Unnamed: 1': 'Estado', 'Unnamed: 2': 'Sexo', 'Unnamed: 3': 'Edad'})
mexico['Estado'] = mexico['Estado'].apply(lambda state: state[:-1])
mexico['Sexo'] = mexico['Sexo'].apply(lambda state: state[:-1])
mexico.to_csv('data/covidmx.csv')


In [20]:
df = pd.read_csv('data/covidworld.csv')
df = df.drop('SNo', axis=1)
def get_country(country_name):
    data = df[df['Country/Region']==country_name]
    country = data.groupby('ObservationDate')
    country = country.sum()[1:]
    return country

mxts = get_country('Mexico')
mxts 

,Confirmed,Deaths,Recovered
ObservationDate,,,
02/28/2020,1.0,0.0,0.0
02/29/2020,4.0,0.0,0.0
03/01/2020,5.0,0.0,0.0
03/02/2020,5.0,0.0,0.0
03/03/2020,5.0,0.0,1.0
03/04/2020,5.0,0.0,1.0
03/05/2020,5.0,0.0,1.0
03/06/2020,6.0,0.0,1.0
03/07/2020,6.0,0.0,1.0


In [21]:
statests = pd.DataFrame()
statests.index.name='Observation date'
for i in range(0,mxts['Confirmed'].size):
    upbound = mxts['Confirmed'][i].astype(int)
    day = pd.DataFrame(index=np.repeat(mxts.index[i],mexico[:upbound]['Estado'].unique().size))
    
    states = mexico[:upbound].groupby('Estado')
    states = states['Estado'].count()
    day['Province/State'] = states.index.values
    day['Confirmed'] = states.values
    statests = statests.append(day)

statests.to_csv('data/covidstates.csv')

In [22]:
states = mexico.groupby('Estado')
infected = states['Estado'].count()
infected = infected.rename({'CIUDAD DE MÉXICO': 'DISTRITO FEDERAL', 'MÉXICO': 'MEXICO', 'NUEVO LEÓN':'NUEVO LEON', 'SAN LUIS POTOSÍ':'SAN LUIS POTOSI'})
infected = infected.sort_index()
infected.to_csv('data/infected.csv')
infected 

Estado
AGUASCALIENTES          48
BAJA CALIFORNIA         52
BAJA CALIFORNIA SUR     21
CAMPECHE                 6
CHIAPAS                 18
CHIHUAHUA               12
COAHUILA                70
COLIMA                   4
DISTRITO FEDERAL       384
DURANGO                  8
GUANAJUATO              52
GUERRERO                24
HIDALGO                 30
JALISCO                116
MEXICO                 177
MICHOACÁN               24
MORELOS                 12
NAYARIT                  8
NUEVO LEON              85
OAXACA                  22
PUEBLA                 109
QUERETARO               35
QUINTANA ROO            65
SAN LUIS POTOSI         35
SINALOA                 51
SONORA                  22
TABASCO                 73
TAMAULIPAS              16
TLAXCALA                 8
VERACRUZ                35
YUCATÁN                 59
ZACATECAS                7
Name: Estado, dtype: int64

In [23]:
states = gpd.read_file('data/geopandas/states.shp')
infected = pd.read_csv('data/infected.csv',header=None)
infected = infected.set_index(0)
infected.index=states.index
states['infected'] = infected
states.to_file('data/geopandas/states.shp')

In [24]:
# path = 'data/PHLITL_2000.shp'
# data = gpd.read_file(path)
# counties = data[['EDO_LEY', 'geometry']]
# states = counties.dissolve(by='EDO_LEY')
# states = states.sort_index()

In [9]:
df = pd.read_csv('data/recursosmx.csv', encoding = "ISO-8859-1")
df

,AÑO,CLUES,SECTOR,CLAVE INSTITUCIÓN,Clave Estado,Nombre Estado,Clave Municipio,Nombre Municipio,Clave Localidad,Nombre Localidad,...,Total otro personal,Número de Personal administrativo,Número de Personal en archivo clínico,Número de Personal en conservación y mantenimiento,Número de Personal de Intendencia (incluye lavandería),Número de Otro personal,Ultrasonido,LITOTRIPTORES,Electrocardiógrafos,Electroencefalógrafos
0,2018,ASIMS000016,PÚBLICO,IMS,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,...,275,108,0,33,134,0,1,0,4,0.0
1,2018,ASIMS000021,PÚBLICO,IMS,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,...,253,110,0,35,108,0,3,0,8,1.0
2,2018,ASIMS000033,PÚBLICO,IMS,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,...,48,39,0,2,7,0,0,0,0,0.0
3,2018,ASIMS000045,PÚBLICO,IMS,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,...,73,54,0,3,16,0,1,0,0,0.0
4,2018,ASIMS000050,PÚBLICO,IMS,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,...,55,40,0,3,12,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,2018,ZSSSA013160,PÚBLICO,SSA,32,ZACATECAS,39,RÍO GRANDE,1,RIO GRANDE,...,7,6,0,0,1,0,1,0,2,NaN
22540,2018,ZSSSA013172,PÚBLICO,SSA,32,ZACATECAS,17,GUADALUPE,1,GUADALUPE,...,33,24,0,0,0,9,1,0,2,NaN
22541,2018,ZSSSA013196,PÚBLICO,SSA,32,ZACATECAS,10,FRESNILLO,1,FRESNILLO,...,16,12,0,0,0,4,0,0,0,NaN
22542,2018,ZSSSA013201,PÚBLICO,SSA,32,ZACATECAS,38,PINOS,1,PINOS,...,6,6,0,0,0,0,1,0,1,0.0


In [6]:
df.columns.values

array(['AÑO', 'CLUES', 'SECTOR', 'CLAVE INSTITUCIÓN', 'Clave Estado',
       'Nombre Estado', 'Clave Municipio', 'Nombre Municipio',
       'Clave Localidad', 'Nombre Localidad', 'Nombre de la Unidad',
       'Tipo de Establecimiento', 'Tipología', 'Número de unidades',
       '¿Cuenta con agua potable?', '¿Cuenta con red municipal?',
       '¿Cuenta con pozo?', '¿Cuenta con cisterna?',
       '¿Cuenta con otra fuente?', '¿Cuenta con drenaje?',
       '¿Cuenta con planta de emergencia? (antes planta eléctrica)',
       '¿Cuenta con radio (banda civil)?',
       '¿Hay Internet en su unidad de salud?',
       '¿Cuenta con correo electrónico (Internet)?',
       '¿Cuenta con red de voz?', '¿Cuenta con red de datos?',
       '¿Cuenta con sistema de telemedicina?',
       '¿Cuenta con expediente clínico electrónico?',
       'Número de Baños Públicos', 'Número de Baños para el personal',
       'Número de Baños para pacientes',
       'Número de Baños para discapacitados',
       '¿Cuenta c

In [13]:
df['Número de camas en la Unidad de Cuidados Intensivos (incluye pediátricas y adulto)'].sum()
df['CLAVE INSTITUCIÓN'].unique()

array(['IMS', 'ISSSTE', 'SSA', 'IMSS PROSPERA', 'SDN', 'SMA', 'SME',
       'PEMEX', 'SMM', 'HUN', 'DIF'], dtype=object)

In [53]:
gb = df.groupby('CLAVE INSTITUCIÓN')
ims = gb.get_group('IMS')

In [55]:
(ims['TOTAL CAMAS AREA HOSPITALIZACIÓN'].sum()+ims['TOTAL CAMAS EN OTRAS AREAS (NO CONSIDERA HOSPITALIZACIÓN)'].sum())*.144

7099.343999999999